# Planejado vs Real vs Projeção

## Prototipo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.interpolate import interp1d

# --- Função Planejado ---
def f(x):
    return 0.6 * (10**(x/14))

# --- Pontos Planejado ---
x_curve = np.linspace(0, 14, 430)
y_curve = f(x_curve)

x_points = np.arange(0, 15, 1)
y_plan = f(x_points)

# Ajustar soma para ~42
target_total = 43
scale_factor = target_total / np.sum(y_plan)
y_curve *= scale_factor
y_plan *= scale_factor

# Quadrante 0 = ponto zero (0kg)
y_plan[0] = 0
accum_plan = np.cumsum(y_plan)

# Criar função de interpolação para a curva suave
accum_curve = np.cumsum(y_curve) * (accum_plan[-1] / np.sum(y_curve))

# --- Dados Reais ---
peso_inicial = 112.4  # Peso inicial em kg
y_real = [0, 1.7, -1.3, 2, -1.1, 2.8, -0.7]  # Valores reais de perda por quadrante
accum_real = np.cumsum(y_real)
x_real = np.arange(len(y_real))  # Corrigido para usar índices reais

# Data de início (ajuste conforme necessário)
data_inicio = datetime(2025, 7, 20)  # Exemplo: 20/07/25

# --- Projeção Corrigida ---
# Encontrar o ponto exato na curva suave que corresponde ao acumulado real
x_proj = []
y_proj = []
ponto_esperado = None
if len(y_real) > 0:
    last_real_value = accum_real[-1]
    
    # Encontrar o quadrante exato onde o planejado tem o valor acumulado de 1.3kg
    # Usando interpolação na curva suave
    f_interp = interp1d(accum_curve, x_curve, kind='linear', fill_value="extrapolate")
    x_esperado = f_interp(last_real_value)
    
    # Criar pontos para a projeção até este ponto
    mask = x_curve <= x_esperado
    x_proj = x_curve[mask]
    y_proj = accum_curve[mask]
    
    ponto_esperado = (x_esperado, last_real_value)

# --- Calcular dias acumulados por fase ---
dias_por_quadrante = [0]  # Quadrante 0 = dia 0
dias_acumulados = [0]     # Acumulado inicia em 0
datas_formatadas = [data_inicio.strftime('%d/%m/%y')]  # Formato dd/mm/yy

# Fase verde (quadrantes 1-3): 5 dias por quadrante
for i in range(1, 4):  # Quadrantes 1 a 3
    dias_por_quadrante.append(5)
    dias_acumulados.append(dias_acumulados[-1] + 5)
    nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
    datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

# Fase azul (quadrantes 4-7): 7 dias por quadrante
for i in range(4, 8):  # Quadrantes 4 a 7
    dias_por_quadrante.append(7)
    dias_acumulados.append(dias_acumulados[-1] + 7)
    nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
    datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

# Fase roxa (quadrantes 8-14): 10 dias por quadrante
for i in range(8, 15):  # Quadrantes 8 a 14
    dias_por_quadrante.append(10)
    dias_acumulados.append(dias_acumulados[-1] + 10)
    nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
    datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

# --- PLOTAGEM ---
plt.figure(figsize=(16, 9))
ax = plt.gca()

# Fases coloridas no fundo com dias calculados
ax.axvspan(-0.5, 3, color='green', alpha=0.2, lw=0)  # Quadrantes 0-3
ax.axvspan(3, 7, color='blue', alpha=0.2, lw=0)   # Quadrantes 4-7
ax.axvspan(7, 14.5, color='purple', alpha=0.2, lw=0) # Quadrantes 8-14

# Adicionar rótulos de fases (acima do gráfico)
max_y = max(accum_plan.max(), accum_real.max() if len(y_real) > 0 else 0) + 5
plt.text(1.5, max_y, '5 dias/quad', ha='center', va='bottom', 
         color='darkgreen', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))
plt.text(5, max_y, '7 dias/quad', ha='center', va='bottom',
         color='darkblue', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))
plt.text(10.5, max_y, '10 dias/quad', ha='center', va='bottom',
         color='purple', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))

# Projeção Corrigida: apenas até o ponto exato na curva planejada
if len(y_proj) > 0 and ponto_esperado is not None:
    plt.plot(x_proj, y_proj, color='blue', alpha=0.4, linewidth=8, zorder=1, label='Projeção')
    # Marcar o ponto onde o planejado atinge o acumulado real
    plt.plot(ponto_esperado[0], ponto_esperado[1], 'bo', markersize=10, zorder=4)
    
    # Formatar rótulo com quadrante aproximado
    quad_aprox = f'Q{ponto_esperado[0]:.1f}'
    plt.text(ponto_esperado[0], ponto_esperado[1] + 2, f'{quad_aprox}: {ponto_esperado[1]:.1f}kg',
             color='blue', fontsize=10, ha='center', va='bottom', weight='bold', zorder=3)

# Planejado (linha pontilhada)
plt.plot(x_points, accum_plan, color='blue', linestyle='--', linewidth=2, label='Planejado', zorder=2)

# Rótulos na linha azul (valores por quadrante)
for i, val in enumerate(y_plan):
    if i == 0:
        continue
    ha = 'left' if i == 1 else ('right' if i == 14 else 'center')
    plt.text(x_points[i], accum_plan[i] + 0.3, f'{val:.1f}',
             color='blue', fontsize=8, ha=ha, va='bottom', weight='bold', zorder=3)

# Real
if len(y_real) > 0:
    plt.plot(x_real, accum_real, color='red', linewidth=2, marker='o', label='Real', zorder=3)
    for i, val in enumerate(accum_real):
        plt.text(x_real[i], val + 0.3, f'{val:.1f}', color='red', fontsize=8,
                 ha='center', va='bottom', weight='bold', zorder=3)

# --- Eixo X com rótulos principais ---
x_tick_labels = []
for i in range(len(accum_plan)):
    if i == 0:
        # Quadrante 0 mostra peso inicial
        main_label = f'Q{i}\nP:0.0/{peso_inicial:.1f}\nD:0'
    else:
        # Outros quadrantes mostram perda acumulada e peso atual projetado
        peso_projetado = peso_inicial - accum_plan[i]
        main_label = f'Q{i}\nP:{accum_plan[i]:.1f}/{peso_projetado:.1f}\nD:{dias_acumulados[i]}'
    x_tick_labels.append(main_label)

ax.set_xticks(x_points)
ax.set_xticklabels(x_tick_labels, fontsize=9)

# Adicionar rótulos de data (dd/mm/yy)
for i in range(1, len(x_points)):
    # Ajuste da posição vertical para evitar o warning
    ax.text(x_points[i] - 0.2, -7.0, datas_formatadas[i], 
            rotation=30, ha='left', va='top', fontsize=10, color='black')

# --- Configurações finais ---
plt.ylabel('Perda de Peso Acumulada (kg)')
plt.title(f'Evolução do Peso - Início: {peso_inicial}kg')
plt.xlim(-0.5, 14.5)
plt.ylim(-3, max_y)
plt.grid(True, linestyle='--', alpha=0.6, color='gray')
plt.legend()
plt.tight_layout()
plt.show()

# Beta Version

Calculadora de Divisão Fibonacci

In [ ]:
import math

def fibonacci_sequence(n, min_val=1):
    """Gera sequência Fibonacci até atingir n valores, começando no primeiro valor >= min_val."""
    if n <= 0:
        return []
    a, b = 1, 1
    sequence = []
    while a < min_val:
        a, b = b, a + b
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b
    return sequence

def redistribute_small_values(block, subblocks):
    """Redistribui valores <=5 combinando-os com elementos adjacentes."""
    if all(x > 5 for x in subblocks) or len(subblocks) == 1:
        return subblocks
    
    result = []
    temp = 0
    for val in subblocks:
        if val <= 5:
            temp += val
        else:
            if temp > 0:
                if result:
                    result[-1] += temp
                else:
                    result.append(temp)
                temp = 0
            result.append(val)
    
    if temp > 0:
        if result:
            result[-1] += temp
        else:
            result.append(temp)
    
    if result and result[0] <= 5 and len(result) > 1:
        result[1] += result[0]
        result = result[1:]
    
    # CORREÇÃO AQUI: 're250sult' foi corrigido para 'result'
    if any(x <= 5 for x in result) and len(result) > 1:
        return redistribute_small_values(block, result)
    
    if sum(result) != block:
        diff = block - sum(result)
        result[-1] += diff
    
    return result

def adjust_blocks(blocks, min_val=6):
    """Garante que todos os blocos principais tenham pelo menos min_val."""
    total = sum(blocks)
    n = len(blocks)
    
    if total < min_val * n:
        return blocks
    
    adjusted = blocks[:]
    
    for i in range(n-1, -1, -1):
        if adjusted[i] < min_val:
            needed = min_val - adjusted[i]
            for j in range(i):
                if adjusted[j] > min_val and adjusted[j] >= needed:
                    adjusted[j] -= needed
                    adjusted[i] = min_val
                    break
            else:
                if adjusted[0] >= needed:
                    adjusted[0] -= needed
                    adjusted[i] = min_val
    
    if all(x >= min_val for x in adjusted):
        return adjusted
    
    for i in range(n-1, -1, -1):
        if adjusted[i] < min_val:
            needed = min_val - adjusted[i]
            for j in range(n):
                if j != i and adjusted[j] >= needed:
                    adjusted[j] -= needed
                    adjusted[i] += needed
                    break
    
    return adjusted

def calculate_blocks(total, num_blocks, num_subblocks):
    fib_blocks = fibonacci_sequence(num_blocks, min_val=2)
    sum_fib_blocks = sum(fib_blocks)
    
    blocks = [round((f/sum_fib_blocks) * total) for f in fib_blocks]
    
    diff = total - sum(blocks)
    blocks[-1] += diff
    
    blocks.sort(reverse=True)
    blocks = adjust_blocks(blocks, min_val=6)
    
    subblocks_table = []
    for i, block in enumerate(blocks):
        current_subblocks = max(1, num_subblocks - i)
        min_sub_val = 1 if block < 15 else 2
        fib_subs = fibonacci_sequence(current_subblocks, min_val=min_sub_val)
        
        sum_fib_subs = sum(fib_subs) or 1
        subblocks = [round((f/sum_fib_subs) * block) for f in fib_subs]
        
        diff = block - sum(subblocks)
        if subblocks:
            subblocks[-1] += diff
        
        if len(subblocks) > 1:
            subblocks = redistribute_small_values(block, subblocks)
        
        subblocks.sort(reverse=True)
        subblocks_table.append(subblocks)
    
    return blocks, subblocks_table

def automatic_training(total):
    """Calcula automaticamente os parâmetros para um treino baseado no total de repetições."""
    # Definir número de blocos baseado no total
    if total <= 50:
        num_blocks = 3
    elif total <= 100:
        num_blocks = 4
    elif total <= 200:
        num_blocks = 5
    elif total <= 300:
        num_blocks = 6
    else:
        num_blocks = 7
    
    # Definir número de subblocos baseado no total
    if total <= 50:
        num_subblocks = 3
    elif total <= 100:
        num_subblocks = 4
    elif total <= 200:
        num_subblocks = 5
    elif total <= 300:
        num_subblocks = 6
    else:
        num_subblocks = 7
    
    print(f"Modo automático: {total} repetições, {num_blocks} blocos, {num_subblocks} subblocos")
    return calculate_blocks(total, num_blocks, num_subblocks)

def format_output(total, blocks, subblocks):
    print(f"\nDESAFIO {total}x")
    for i in range(len(blocks)):
        sub_str = ', '.join(map(str, subblocks[i]))
        print(f"{blocks[i]}x = [{sub_str}]")

# Entrada interativa
if __name__ == "__main__":
    print("Calculadora de Divisão Fibonacci")
    
    total = int(input("Valor total de repetições (0 para modo automático): "))
    
    if total == 0:
        auto_total = int(input("Total de repetições para o treino automático: "))
        blocks, subblocks = automatic_training(auto_total)
        format_output(auto_total, blocks, subblocks)
    else:
        num_blocks = int(input("Número de blocos principais: "))
        num_subblocks = int(input("Número de subblocos por bloco: "))
        blocks, subblocks = calculate_blocks(total, num_blocks, num_subblocks)
        format_output(total, blocks, subblocks)

Calculadora de Divisão Fibonacci


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def fibonacci_proportion(n, start=2):
    """Gera proporções de Fibonacci"""
    a, b = 1, 1
    sequence = []
    while a < start:
        a, b = b, a + b
    for _ in range(n):
        sequence.append(a)
        a, b = b, a + b
    return np.array(sequence) / sum(sequence)

# Exemplo para 5 blocos e 3 níveis hierárquicos
T = 120
n_blocks = 5
n_levels = 3

# Criar matriz de valores
matrix = np.zeros((n_blocks, n_levels))

# Preencher com proporções de Fibonacci
for i in range(n_blocks):
    # Proporções para este nível
    props = fibonacci_proportion(n_levels, start=max(2, n_levels-i))
    
    # Calcular valores (simplificado)
    if i == 0:
        matrix[i] = props * T
    else:
        matrix[i] = props * matrix[i-1, 0]

# Plotar em 3D
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

x = np.arange(n_blocks)
y = np.arange(n_levels)
X, Y = np.meshgrid(x, y)
Z = matrix.T  # Transpor para compatibilidade com meshgrid

ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax.set_xlabel('Posição na Sequência')
ax.set_ylabel('Nível Hierárquico')
ax.set_zlabel('Valor')
ax.set_title('Distribuição Fibonacci de Treino')

plt.show()

## **Sem a fase vermelha**
ou seja, com desafio de no maximo 10 dias entre q1 e 14 e 15 dias para fase preta

### Prototipo de Plano de queima calorica

In [ ]:
import numpy as np
from datetime import datetime, timedelta

# ======================
# CONSTANTES GLOBAIS
# ======================

# GASTO CALÓRICO POR ATIVIDADE
KCAL_KARATE = 1000      # 120 minutos
KCAL_CALISTENIA = 300   # 100 repetições
KCAL_CORRIDA = 100      # por km
KCAL_BIKE = KCAL_CORRIDA/3    # por km (3km = 100kcal)
KCAL_AGUA = 0           # apenas para registro

# CONSTANTES METABÓLICAS
BASAL = 2300            # Taxa metabólica basal
KCAL_POR_KG = 7000      # 7000kcal = 1kg
AGUA_Q10 = 5000         # 5L de água no Q10
MAX_DEFICIT_ALIMENTAR = 1300  # Déficit máximo via alimentação
DIAS_EMERGENCIA = 15    # Duração padrão da fase preta

# ======================
# FUNÇÕES AUXILIARES
# ======================

def f(x):
    return 0.6 * (10**(x/14))

# ======================
# CONFIGURAÇÃO INICIAL
# ======================

x_points = np.arange(0, 21, 1)
y_plan = f(x_points)

target_total = 43
scale_factor = target_total / np.sum(y_plan[:15])
y_plan *= scale_factor
y_plan[0] = 0

dias_por_quadrante = [0]
for i in range(1, 3): dias_por_quadrante.append(5)
for i in range(3, 6): dias_por_quadrante.append(7)
for i in range(6, 15): dias_por_quadrante.append(10)
for i in range(15, 21): dias_por_quadrante.append(DIAS_EMERGENCIA)

# ======================
# FUNÇÃO PRINCIPAL
# ======================

def calcular_plano(quadrante, meta_kg_emergencia=None, dias_emergencia=None, data_inicio_emergencia=None):
    if quadrante < 0 or quadrante > 20:
        raise ValueError("Quadrante deve estar entre 0 e 20")
    
    data_inicio = datetime(2025, 7, 20)
    
    if quadrante >= 15 and data_inicio_emergencia:
        data_inicio_quad = datetime.strptime(data_inicio_emergencia, "%d/%m/%Y")
    else:
        dias_anteriores = sum(dias_por_quadrante[:quadrante])
        data_inicio_quad = data_inicio + timedelta(days=dias_anteriores)
    
    if quadrante <= 14:
        meta_kg = y_plan[quadrante]
        dias = dias_por_quadrante[quadrante]
    else:
        if meta_kg_emergencia is None or dias_emergencia is None:
            raise ValueError("Para fase de emergência (Q15+), informe a meta de kg a perder e dias")
        meta_kg = meta_kg_emergencia
        dias = dias_emergencia
    
    total_kcal = meta_kg * KCAL_POR_KG
    total_kcal = np.ceil(total_kcal / 500) * 500
    
    deficit_diario = total_kcal / dias
    deficit_diario = np.ceil(deficit_diario / 500) * 500
    
    if quadrante <= 2:
        fase = "verde"
    elif quadrante <= 5:
        fase = "azul"
    elif quadrante <= 14:
        fase = "roxo"
    else:
        fase = "PRETO (emergência)"
    
    if quadrante <= 14:
        reducao_dieta = (MAX_DEFICIT_ALIMENTAR / 14) * quadrante
        deficit_alimentacao = min(reducao_dieta, MAX_DEFICIT_ALIMENTAR)
        deficit_alimentacao = round(deficit_alimentacao, 1)
    else:
        deficit_alimentacao = MAX_DEFICIT_ALIMENTAR
    
    meta_consumo = BASAL - deficit_alimentacao
    queima_necessaria = max(0, deficit_diario - deficit_alimentacao)

    if quadrante >= 15:
        queima_necessaria *= 1.5

    datas = [data_inicio_quad + timedelta(days=i) for i in range(dias)]
    dias_semana = [data.strftime("%A") for data in datas]
    
    dias_trad = {
        "Monday": "segunda", "Tuesday": "terça", "Wednesday": "quarta",
        "Thursday": "quinta", "Friday": "sexta", "Saturday": "sábado",
        "Sunday": "domingo"
    }
    dias_semana = [dias_trad[dia] for dia in dias_semana]
    
    karate_dias = [i for i, dia in enumerate(dias_semana) if dia in ["segunda", "quarta"]]
    
    if quadrante >= 15:
        agua_diaria = 6000
    else:
        agua_diaria = min(int((AGUA_Q10 / 10) * quadrante), AGUA_Q10)
    
    planos = []
    for i, data in enumerate(datas):
        dia_semana = dias_semana[i]
        queima_dia = queima_necessaria
        plano = []
        
        # 1. EXERCÍCIOS BASE (Calistenia + Corrida + Bike)
        reps = quadrante * 10 if quadrante > 0 else 15 * 10  # Usa Q15 como base para emergência
        kcal_calistenia = (reps / 100) * KCAL_CALISTENIA
        plano.append(("Calistenia", f"{reps} reps", kcal_calistenia))
        
        km_corrida = quadrante if quadrante > 0 else 15  # Usa Q15 como base para emergência
        kcal_corrida = km_corrida * KCAL_CORRIDA
        plano.append(("Corrida", f"{km_corrida} km", kcal_corrida))
        
        # Bike sempre 3x a corrida (regra fixa)
        km_bike = (quadrante if quadrante > 0 else 15) * 3
        kcal_bike = km_bike * KCAL_BIKE
        plano.append(("Bike", f"{km_bike} km", kcal_bike))
        
        # 2. KARATE OBRIGATÓRIO (seg/qua a partir do Q3)
        if (quadrante >= 3 or quadrante == 0) and dia_semana in ["segunda", "quarta"]:
            minutos_karate = 120  # Mínimo obrigatório
            kcal_karate = KCAL_KARATE
            plano.append(("Karate Obrigatório", f"{minutos_karate} min", kcal_karate))
            
            # Calcula o excesso que o karate vai causar
            total_exerc = sum([x[2] for x in plano])
            deficit_previsto = total_exerc + deficit_alimentacao
            
            # Ajusta o déficit alimentar para compensar o karate
            if deficit_previsto > deficit_diario:
                excesso = deficit_previsto - deficit_diario
                deficit_alimentacao_ajustado = max(0, deficit_alimentacao - excesso)
            else:
                deficit_alimentacao_ajustado = deficit_alimentacao
        else:
            deficit_alimentacao_ajustado = deficit_alimentacao

        # 3. KARATE OPCIONAL (outros dias a partir do Q3)
        if (quadrante >= 3 or quadrante == 0) and dia_semana not in ["segunda", "quarta"] and queima_dia > 0:
            # Calcula minutos proporcionais ao déficit restante
            minutos_karate = min(120, (queima_dia / KCAL_KARATE) * 120)
            if minutos_karate >= 30:  # Mínimo de 30 minutos
                kcal_karate = (minutos_karate / 120) * KCAL_KARATE
                plano.append(("Karate Opcional", f"{minutos_karate:.0f} min", kcal_karate))
        
        # Cálculo final
        total_exerc = sum([x[2] for x in plano])
        total_deficit = total_exerc + deficit_alimentacao_ajustado
        
        # 4. AJUSTE FINAL COM BIKE EXTRA (se necessário)
        if total_deficit < deficit_diario:
            deficit_restante = deficit_diario - total_deficit
            km_bike_extra = deficit_restante / KCAL_BIKE
            kcal_bike_extra = km_bike_extra * KCAL_BIKE
            
            # Atualiza bike no plano
            for idx, (nome, det, kcal) in enumerate(plano):
                if nome == "Bike":
                    km_total = km_bike + km_bike_extra
                    plano[idx] = ("Bike", f"{km_total:.1f} km", kcal + kcal_bike_extra)
                    break
            
            # Recalcula totais após ajuste
            total_exerc = sum([x[2] for x in plano])
            total_deficit = total_exerc + deficit_alimentacao_ajustado
        
        planos.append((
            data.strftime("%d/%m/%Y"),
            dia_semana,
            plano,
            total_exerc,
            deficit_alimentacao_ajustado,
            total_deficit,
            deficit_diario
        ))

    return {
        "quadrante": quadrante if quadrante > 0 else 15,  # Mostra como Q15 no output
        "fase": fase,
        "dias": dias,
        "inicio": data_inicio_quad.strftime("%d/%m/%Y"),
        "meta_kg": round(meta_kg, 2),
        "total_kcal": total_kcal,
        "deficit_diario": deficit_diario,
        "deficit_alimentacao": deficit_alimentacao,
        "queima_necessaria": queima_necessaria,
        "karate_dias": len(karate_dias),
        "agua": agua_diaria,
        "planos": planos
    }

# ======================
# EXECUÇÃO PRINCIPAL
# ======================

if __name__ == "__main__":
    try:
        quadrante = int(input("\nInforme o quadrante atual (0 para emergência customizada): "))
        
        if quadrante == 0:
            print("\n🚨 PROTOCOLO DE EMERGÊNCIA ATIVADO 🚨")
            hoje = datetime.now().strftime("%d/%m/%Y")
            print(f"📅 Data de início: {hoje} (hoje)")
            
            meta_kg_emergencia = float(input("Informe a meta de kg a perder: "))
            dias_emergencia = int(input("Informe o número de dias para atingir a meta: "))
            data_inicio_emergencia = hoje
        elif 1 <= quadrante <= 14:
            meta_kg_emergencia = None
            dias_emergencia = None
            data_inicio_emergencia = None
        else:
            raise ValueError("Quadrante inválido. Deve ser entre 1-14 ou 0 para emergência.")
        
        res = calcular_plano(
            quadrante if quadrante > 0 else 15,  # Trata Q0 como Q15 internamente
            meta_kg_emergencia,
            dias_emergencia,
            data_inicio_emergencia
        )
        
        print(f"\n{'='*70}")
        print(f"📊 QUADRANTE: {res['quadrante']} | FASE: {res['fase'].upper()}")
        print(f"⏳ Dias: {res['dias']} (início: {res['inicio']})")
        print(f"🎯 Meta: {res['meta_kg']} kg | {res['total_kcal']} kcal")
        print(f"🔥 Déficit diário necessário: {res['deficit_diario']} kcal")
        
        if quadrante == 0 or res['quadrante'] >= 15:
            print("🚨 PROTOCOLO DE EMERGÊNCIA ATIVADO 🚨")
        print(f"🍎 Dieta Déficit base: {res['deficit_alimentacao']} kcal")
            
        print(f"🥋 Dias de Karate: {res['karate_dias']}")
        print(f"💧 Água: {res['agua']}ml")
        
        print("\n📅 PLANO DIÁRIO:")
        for data, dia, plano, exerc, aliment, total, meta in res['planos']:
            print(f"\n>>> {data} ({dia}):")
            for nome, det, kcal in plano:
                print(f" - {nome}: {det} = {kcal:.1f} kcal")
            
            status = "✅ ATINGIDO" if total >= meta else f"⚠️ FALTAM {meta - total:.1f} kcal"
            print(f"🍎 Dieta Déficit: {aliment:.1f} kcal")
            print(f"🔥 TOTAL: {total:.1f} kcal {status}")
            print(f"   (Exerc. {exerc:.1f} + Alim. {aliment:.1f} | Meta: {meta} kcal)")
        
    except ValueError as e:
        print(f"Erro: {e}")

# Teste v1

### Regras de Negócio do Plano Diário de Queima Calórica

1. **Fases e Duração:**
   - **Verde (Q1-Q3):** 5 dias/quadrante
   - **Azul (Q4-Q7):** 7 dias/quadrante
   - **Roxa (Q8-Q14):** 10 dias/quadrante
   - **Preta (Q15+):** 15 dias/quadrante (emergência)
    --remova a fazer preta, e mantenha so o Q0 do protocolo de emergencia

2. **Atividades Base Diárias:**
   - **Calistenia:** `10 reps × quadrante` (ex: Q5 = 50 reps)
   - **Corrida:** `1 km × quadrante`
   - **Bike:** `3 km × quadrante` (sempre 3× a corrida)

3. **Karate:**
   - **Obrigatório:** Segundas/quartas a partir do Q3 (120 min = 1000 kcal)
   - **Opcional:** Sexta e Domingos se déficit não atingido (valor proximo ou igual de um treino regular que aonctece na seg/qrta )

4. **Cálculo de Déficit:**
   ```python
   meta_kg = f(quadrante)  # Função exponencial
   total_kcal = meta_kg × 7000  # 7000kcal = 1kg
-- primeiro: distribuir os treinos de calistenia, corrida, e bike igualmente por todos os dias que foram planejados para quadrante
-- Obdecer a parte que diz a respeito sobre se estamos em Q4, logo é 40% de Q10, se eh Q5, 50% de Q10, se Eh Q12, 120% de Q10, e assim por diante.
-- se a meta atual de Q5 eh perder 1.2kg, a logica anterior geralmente é o suficiente para caluclar as calorias necessarias.
    --exceção, levar em consideracao a linha vermelha, de dados reais. se a meta estiver abaixo do planejado, deve se acrescentar execicios extas para o desafio atual,
ex: meta de Q5 é 1.6, porem a meta real ta 3g abaixo do experado, logo, os execicios extras serao de 3x7000 calorias, mas meta Q5, totalizando, um deficit de 4.6 a ser atingido em Q5




   deficit_diario = arredondar_500(total_kcal / dias)
   deficit_alimentar = min(1300, (1300/14)*quadrante)  # Progressivo até Q14
   queima_necessaria = max(0, deficit_diario - deficit_alimentar)
   ```

5. **Ajustes Dinâmicos:**
   - Se déficit não atingido: **Bike extra** para completar
   - Emergência (Q15+): Queima necessária multiplicada por 1.5

6. **Hidratação:**
   - Progressiva até Q10: `500 ml × quadrante` (max 5000ml)
   - Emergência: 6000ml fixo

7. **Regras Especiais:**
   - **Q0 (emergência custom):** Usa padrão Q15 com inputs do usuário
   - **Arredondamentos:** Sempre múltiplos de 500 kcal
   - **Dias de Treino:** Karate só aparece a partir do Q3

### Fluxo de Cálculo Diário:
```mermaid
graph TD
    A[Início do Dia] --> B[Atividades Base]
    B --> C{É seg/qua?}
    C -->|Sim| D[Karate Obrigatório 120min]
    C -->|Não| E{Precisa queimar mais?}
    E -->|Sim| F[Karate Opcional]
    F --> G[Calcular Déficit Restante]
    D --> G
    G --> H{Déficit Alcançado?}
    H -->|Não| I[Acrescentar Bike Extra]
    H -->|Sim| J[Plano Finalizado]
    I --> J
```

### Pontos Críticos de Controle:
1. **Progressão Exponencial:** Metas de perda seguem `f(x) = 0.6 × (10^(x/14))`
2. **Consistência Temporal:** Datas calculadas a partir de 20/07/2025
3. **Fator Emergência:** Q15+ aumenta exigência em 50%
4. **Mínimo Efetivo:** Karate opcional só acionado com déficit > 250 kcal
5. **Arredondamento Estratégico:** Valores sempre múltiplos de 500 kcal para facilitar acompanhamento

Essas regras garantem que o plano acompanhe a curva exponencial de perda de peso enquanto adapta as atividades físicas conforme a fase e necessidades diárias de déficit calórico.

In [ ]:
import json
import os

# ======================
# CONSTANTES GLOBAIS
# ======================

# GASTO CALÓRICO POR ATIVIDADE
KCAL_KARATE = 1000      # 120 minutos
KCAL_CALISTENIA = 300   # 100 repetições
KCAL_CORRIDA = 100      # por km
KCAL_BIKE = KCAL_CORRIDA / 3  # per km (3km = 100kcal)

# CONSTANTES METABÓLICAS
BASAL = 2300            # Taxa metabólica basal
KCAL_POR_KG = 7000      # 7000kcal = 1kg
AGUA_Q10 = 5000         # 5L de água no Q10
MAX_DEFICIT_ALIMENTAR = 1300  # Déficit máximo via alimentação

# ======================
# FUNÇÕES DO PRIMEIRO ARQUIVO
# ======================

def f(x):
    """Função de progressão da perda de peso."""
    return 0.6 * (10 ** (x / 14))

def configurar_quadrantes():
    """Configura dias e metas por quadrante."""
    x_points = np.arange(0, 15, 1)
    y_plan = f(x_points)
    
    # Ajustar soma total para ~43kg
    target_total = 43
    scale_factor = target_total / np.sum(y_plan[1:15])
    y_plan *= scale_factor
    y_plan[0] = 0  # Quadrante 0 = 0kg
    
    # Dias por quadrante (Q0 a Q14)
    dias_por_quadrante = [0]  # Q0
    for _ in range(1, 4): dias_por_quadrante.append(5)    # Q1-Q3: verde
    for _ in range(4, 8): dias_por_quadrante.append(7)    # Q4-Q7: azul
    for _ in range(8, 15): dias_por_quadrante.append(10)  # Q8-Q14: roxo
    
    return x_points, y_plan, dias_por_quadrante

# Configura quadrantes uma vez para todo o programa
x_points, y_plan, dias_por_quadrante = configurar_quadrantes()
data_inicio_padrao = datetime(2025, 7, 20)

# ======================
# FUNÇÕES AUXILIARES
# ======================

def arredondar_500(valor):
    """Arredonda valores para o múltiplo de 500 mais próximo."""
    return round(valor / 500) * 500

def determinar_fase(quadrante):
    """Classifica a fase do quadrante."""
    if quadrante <= 3:
        return "verde"
    elif quadrante <= 7:
        return "azul"
    return "roxo"

def calcular_atividades_base(quadrante):
    """Calcula atividades físicas base para o quadrante."""
    # Progressão em relação ao Q10
    fator_q10 = quadrante / 10.0
    
    reps = max(1, round(10 * fator_q10 * 10))
    km_corrida = max(1, round(1 * fator_q10 * 10, 1))
    km_bike = max(1, round(3 * fator_q10 * 10, 1))
    
    return [
        ("Calistenia", f"{reps} reps", (reps / 100) * KCAL_CALISTENIA),
        ("Corrida", f"{km_corrida} km", km_corrida * KCAL_CORRIDA),
        ("Bike", f"{km_bike} km", km_bike * KCAL_BIKE)
    ]

def calcular_meta_karate_diaria(dia_semana, quadrante):
    """Calcula a meta diária de karatê baseada no dia da semana e quadrante."""
    if dia_semana in ["segunda", "quarta"]:
        # Segunda e quarta: 100% do Q10 (120 minutos)
        return 120, KCAL_KARATE
    else:
        # Outros dias: proporcional ao quadrante vigente
        minutos_karate = (quadrante / 10.0) * 120
        kcal_karate = (minutos_karate / 120) * KCAL_KARATE
        return minutos_karate, kcal_karate

def ajustar_karate(dia_semana, quadrante, plano_atual, deficit_restante):
    """Adiciona sessões de karatê conforme necessário."""
    if quadrante < 3:
        return 0, plano_atual
    
    # Calcular meta de karatê para o dia
    meta_minutos, meta_kcal = calcular_meta_karate_diaria(dia_semana, quadrante)
    
    # Karatê obrigatório (seg/qua) - sempre adiciona 120min
    if dia_semana in ["segunda", "quarta"]:
        plano_atual.append(("Karate Obrigatório", "120 min", KCAL_KARATE))
        return KCAL_KARATE, plano_atual
    
    # Para outros dias, adiciona karatê proporcional se necessário para o déficit
    if deficit_restante > 250:
        # Calcula minutos proporcionais ao déficit restante, mas não excede a meta
        minutos_karate = min(meta_minutos, (deficit_restante / KCAL_KARATE) * 120)
        if minutos_karate >= 30:
            kcal_karate = (minutos_karate / 120) * KCAL_KARATE
            tipo_karate = "Karate Diário" if minutos_karate >= meta_minutos * 0.8 else "Karate Opcional"
            plano_atual.append((tipo_karate, f"{minutos_karate:.0f} min", kcal_karate))
            return kcal_karate, plano_atual
    
    return 0, plano_atual

def carregar_dados_reais():
    """Carrega dados reais do arquivo JSON."""
    dados_padrao = {
        "peso_inicial": 112.4,
        "dados_reais": [0, 1.7, -1.3, 2, -1.1, 2.8, -0.7],
        "quadrante_atual": 6,
        "data_inicio": "2025-07-20"
    }
    
    try:
        with open('dados_reais.json', 'r') as f:
            dados = json.load(f)
            if len(dados["dados_reais"]) < 7:
                dados["dados_reais"] = dados_padrao["dados_reais"]
            return dados
    except FileNotFoundError:
        return dados_padrao

def calcular_data_inicio_quadrante(quadrante):
    """Calcula a data de início correta para cada quadrante."""
    dados = carregar_dados_reais()
    
    if isinstance(dados["data_inicio"], str):
        data_inicio = datetime.strptime(dados["data_inicio"], "%Y-%m-%d")
    else:
        data_inicio = data_inicio_padrao
    
    # Calcular dias acumulados até o início do quadrante
    dias_acumulados = sum(dias_por_quadrante[1:quadrante])
    return data_inicio + timedelta(days=dias_acumulados)

def calcular_quadrante_atual():
    """Calcula o quadrante atual com base na data."""
    dados = carregar_dados_reais()
    
    if isinstance(dados["data_inicio"], str):
        data_inicio = datetime.strptime(dados["data_inicio"], "%Y-%m-%d")
    else:
        data_inicio = data_inicio_padrao
    
    data_atual = datetime.now()
    dias_decorridos = (data_atual - data_inicio).days
    
    # Calcular em qual quadrante estamos
    dias_acumulados = 0
    for i, dias_quad in enumerate(dias_por_quadrante):
        if i == 0:  # Pular quadrante 0
            continue
        dias_acumulados += dias_quad
        if dias_decorridos < dias_acumulados:
            return i
    
    return 14

def calcular_desvio_acumulado(quadrante_atual):
    """Calcula o desvio acumulado com base nos dados reais."""
    dados = carregar_dados_reais()
    dados_reais = dados["dados_reais"]
    
    # Calcular perda real acumulada
    perda_real_acumulada = sum(dados_reais)
    
    # Calcular perda planejada acumulada até o quadrante atual
    perda_planejada_acumulada = np.sum(y_plan[1:quadrante_atual+1])
    
    # Calcular desvio (diferença entre planejado e realizado)
    desvio = perda_planejada_acumulada - perda_real_acumulada
    return desvio

def calcular_meta_proximo_quadrante(quadrante_atual):
    """Calcula la meta necessária para o próximo quadrante."""
    # Calcular desvio acumulado
    desvio = calcular_desvio_acumulado(quadrante_atual)
    
    # Meta planejada para o próximo quadrante
    if quadrante_atual + 1 < len(y_plan):
        meta_planejada = y_plan[quadrante_atual + 1]
    else:
        meta_planejada = y_plan[-1]
    
    # Meta ajustada = planejada + desvio acumulado
    meta_ajustada = meta_planejada + desvio
    
    return max(0.1, meta_ajustada)

# ======================
# FUNÇÃO PRINCIPAL COM REGRAS DE KARATÊ CORRIGIDAS
# ======================

def calcular_plano(quadrante, meta_kg=None, dias=None, data_inicio_quad=None):
    """Calcula plano detalhado para o quadrante especificado."""
    # Configurar data de início
    if quadrante == 0:
        fase = "EMERGÊNCIA"
        desvio_acumulado = 0
        if not all([meta_kg, dias, data_inicio_quad]):
            raise ValueError("Para emergência, informe meta_kg, dias e data_inicio_quad")
    else:
        if quadrante < 1 or quadrante > 14:
            raise ValueError("Quadrante deve estar entre 1-14")
            
        fase = determinar_fase(quadrante)
        dias = dias_por_quadrante[quadrante]
        data_inicio_quad = calcular_data_inicio_quadrante(quadrante)
        
        # Se não foi fornecida meta, calcular com base no desvio
        if meta_kg is None:
            desvio_acumulado = calcular_desvio_acumulado(quadrante - 1)
            meta_kg = y_plan[quadrante] + desvio_acumulado
        else:
            desvio_acumulado = meta_kg - y_plan[quadrante]
    
    # Cálculos de déficit
    total_kcal = meta_kg * KCAL_POR_KG
    deficit_diario = arredondar_500(total_kcal / dias)
    
    # Déficit alimentar progressivo
    if quadrante == 0:
        deficit_alimentacao = MAX_DEFICIT_ALIMENTAR
    else:
        deficit_alimentacao = min(MAX_DEFICIT_ALIMENTAR, 
                                 (MAX_DEFICIT_ALIMENTAR / 14) * quadrante)
    
    queima_necessaria = max(0, deficit_diario - deficit_alimentacao)
    
    # Configurar datas e dias da semana
    datas = [data_inicio_quad + timedelta(days=i) for i in range(dias)]
    dias_semana = [data.strftime("%A") for data in datas]
    tradutor_dias = {
        "Monday": "segunda", "Tuesday": "terça", "Wednesday": "quarta",
        "Thursday": "quinta", "Friday": "sexta", "Saturday": "sábado",
        "Sunday": "domingo"
    }
    dias_semana = [tradutor_dias[dia] for dia in dias_semana]
    
    # Calcular plano diário
    planos = []
    for i, data in enumerate(datas):
        # 1. Atividades base
        plano_dia = calcular_atividades_base(quadrante if quadrante > 0 else 5)
        total_exerc = sum(kcal for _, _, kcal in plano_dia)
        deficit_restante = max(0, queima_necessaria - total_exerc)
        
        # 2. Ajustar karatê
        kcal_karate, plano_dia = ajustar_karate(
            dias_semana[i], 
            quadrante, 
            plano_dia,
            deficit_restante
        )
        total_exerc += kcal_karate
        deficit_restante = max(0, queima_necessaria - total_exerc)
        
        # 3. Calcular meta de karate para o dia
        meta_minutos, meta_kcal = calcular_meta_karate_diaria(dias_semana[i], quadrante)
        
        # 4. Verificar se o karate atingiu a meta antes de adicionar bike extra
        karate_atingiu_meta = kcal_karate >= meta_kcal * 0.8  # 80% da meta considera atingido
        
        # 5. Ajustar bike apenas se karate atingiu a meta
        atividades_complementares = []
        if deficit_restante > 0 and karate_atingiu_meta:
            # Encontra atividade de bike para atualizar
            for idx, (nome, det, kcal) in enumerate(plano_dia):
                if nome == "Bike":
                    km_extra = deficit_restante / KCAL_BIKE
                    km_total = float(det.split()[0]) + km_extra
                    kcal_extra = km_extra * KCAL_BIKE
                    plano_dia[idx] = ("Bike", f"{km_total:.1f} km", kcal + kcal_extra)
                    atividades_complementares.append(("Bike Extra", f"+{km_extra:.1f} km", kcal_extra))
                    total_exerc += kcal_extra
                    break
        
        # 6. Calcular déficit total
        total_deficit = total_exerc + deficit_alimentacao
        
        planos.append((
            data.strftime("%d/%m/%Y"),
            dias_semana[i],
            plano_dia,
            atividades_complementares,
            total_exerc,
            deficit_alimentacao,
            total_deficit,
            deficit_diario
        ))

    return {
        "quadrante": quadrante,
        "fase": fase,
        "dias": dias,
        "inicio": data_inicio_quad.strftime("%d/%m/%Y"),
        "meta_kg": round(meta_kg, 2),
        "desvio_kg": round(desvio_acumulado, 2) if quadrante > 0 else 0,
        "total_kcal": total_kcal,
        "deficit_diario": deficit_diario,
        "deficit_alimentacao": deficit_alimentacao,
        "queima_necessaria": queima_necessaria,
        "agua": min(int(500 * quadrante), AGUA_Q10) if quadrante > 0 else 6000,
        "planos": planos
    }

# ======================
# FUNÇÃO DE PLOTAGEM
# ======================

def plotar_evolucao_peso():
    """Função para plotar a evolução do peso."""
    # Pontos Planejado
    x_curve = np.linspace(0, 14, 430)
    y_curve = f(x_curve)

    # Ajustar soma para ~43kg
    target_total = 43
    scale_factor = target_total / np.sum(y_plan[1:])
    y_curve *= scale_factor
    y_plan_adjusted = y_plan.copy()

    # Quadrante 0 = ponto zero (0kg)
    y_plan_adjusted[0] = 0
    accum_plan = np.cumsum(y_plan_adjusted)

    # Criar função de interpolação para a curva suave
    accum_curve = np.cumsum(y_curve) * (accum_plan[-1] / np.sum(y_curve))

    # Dados Reais
    dados = carregar_dados_reais()
    peso_inicial = dados["peso_inicial"]
    y_real = dados["dados_reais"]
    accum_real = np.cumsum(y_real)
    x_real = np.arange(len(y_real))

    # Data de início
    if isinstance(dados["data_inicio"], str):
        data_inicio = datetime.strptime(dados["data_inicio"], "%Y-%m-%d")
    else:
        data_inicio = data_inicio_padrao

    # Projeção Corrigida
    x_proj = []
    y_proj = []
    ponto_esperado = None
    if len(y_real) > 0:
        last_real_value = accum_real[-1]
        
        f_interp = interp1d(accum_curve, x_curve, kind='linear', fill_value="extrapolate")
        x_esperado = f_interp(last_real_value)
        
        mask = x_curve <= x_esperado
        x_proj = x_curve[mask]
        y_proj = accum_curve[mask]
        
        ponto_esperado = (x_esperado, last_real_value)

    # Calcular dias acumulados por fase
    dias_acumulados = [0]
    datas_formatadas = [data_inicio.strftime('%d/%m/%y')]

    for i in range(1, 4):
        dias_acumulados.append(dias_acumulados[-1] + 5)
        nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
        datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

    for i in range(4, 8):
        dias_acumulados.append(dias_acumulados[-1] + 7)
        nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
        datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

    for i in range(8, 15):
        dias_acumulados.append(dias_acumulados[-1] + 10)
        nova_data = data_inicio + timedelta(days=dias_acumulados[-1])
        datas_formatadas.append(nova_data.strftime('%d/%m/%y'))

    # PLOTAGEM
    plt.figure(figsize=(16, 9))
    ax = plt.gca()

    # Fases coloridas no fundo
    ax.axvspan(-0.5, 3, color='green', alpha=0.2, lw=0)
    ax.axvspan(3, 7, color='blue', alpha=0.2, lw=0)
    ax.axvspan(7, 14.5, color='purple', alpha=0.2, lw=0)

    max_y = max(accum_plan.max(), accum_real.max() if len(y_real) > 0 else 0) + 5
    plt.text(1.5, max_y, '5 dias/quad', ha='center', va='bottom', 
             color='darkgreen', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))
    plt.text(5, max_y, '7 dias/quad', ha='center', va='bottom',
             color='darkblue', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))
    plt.text(10.5, max_y, '10 dias/quad', ha='center', va='bottom',
             color='purple', fontsize=9, bbox=dict(facecolor='white', alpha=0.8))

    # Projeção Corrigida
    if len(y_proj) > 0 and ponto_esperado is not None:
        plt.plot(x_proj, y_proj, color='blue', alpha=0.4, linewidth=8, zorder=1, label='Projeção')
        plt.plot(ponto_esperado[0], ponto_esperado[1], 'bo', markersize=10, zorder=4)
        
        quad_aprox = f'Q{ponto_esperado[0]:.1f}'
        plt.text(ponto_esperado[0], ponto_esperado[1] + 2, f'{quad_aprox}: {ponto_esperado[1]:.1f}kg',
                 color='blue', fontsize=10, ha='center', va='bottom', weight='bold', zorder=3)

    # Planejado
    plt.plot(x_points, accum_plan, color='blue', linestyle='--', linewidth=2, label='Planejado', zorder=2)

    # Rótulos na linha azul
    for i, val in enumerate(y_plan_adjusted):
        if i == 0:
            continue
        ha = 'left' if i == 1 else ('right' if i == 14 else 'center')
        plt.text(x_points[i], accum_plan[i] + 0.3, f'{val:.1f}',
                 color='blue', fontsize=8, ha=ha, va='bottom', weight='bold', zorder=3)

    # Real
    if len(y_real) > 0:
        plt.plot(x_real, accum_real, color='red', linewidth=2, marker='o', label='Real', zorder=3)
        for i, val in enumerate(accum_real):
            plt.text(x_real[i], val + 0.3, f'{val:.1f}', color='red', fontsize=8,
                     ha='center', va='bottom', weight='bold', zorder=3)

    # Eixo X com rótulos principais
    x_tick_labels = []
    for i in range(len(accum_plan)):
        if i == 0:
            main_label = f'Q{i}\nP:0.0/{peso_inicial:.1f}\nD:0'
        else:
            peso_projetado = peso_inicial - accum_plan[i]
            main_label = f'Q{i}\nP:{accum_plan[i]:.1f}/{peso_projetado:.1f}\nD:{dias_acumulados[i]}'
        x_tick_labels.append(main_label)

    ax.set_xticks(x_points)
    ax.set_xticklabels(x_tick_labels, fontsize=9)

    # Adicionar rótulos de data
    for i in range(1, len(x_points)):
        ax.text(x_points[i] - 0.2, -7.0, datas_formatadas[i], 
                rotation=30, ha='left', va='top', fontsize=10, color='black')

    # Configurações finais
    plt.ylabel('Perda de Peso Acumulada (kg)')
    plt.title(f'Evolução do Peso - Início: {peso_inicial}kg | Atual: {peso_inicial - accum_real[-1]:.1f}kg')
    plt.xlim(-0.5, 14.5)
    plt.ylim(-3, max_y)
    plt.grid(True, linestyle='--', alpha=0.6, color='gray')
    plt.legend()
    plt.tight_layout()
    plt.show()

# ======================
# FUNÇÃO PRINCIPAL DE EXECUÇÃO
# ======================

def executar_programa_principal():
    """Executa o programa principal."""
    try:
        # Carregar dados atuais
        dados = carregar_dados_reais()
        quadrante_atual = 6  # Forçando Q6 conforme solicitado
        dados_reais = dados["dados_reais"]
        perda_acumulada = sum(dados_reais)
        peso_atual = dados["peso_inicial"] - perda_acumulada
        
        # Calcular desvio atual (até Q5, já que Q6 está em andamento)
        desvio = calcular_desvio_acumulado(5)  # Até Q5
        
        # Calcular perda planejada acumulada até Q5
        perda_planejada_acum = np.sum(y_plan[1:6])
        
        # Calcular perda planejada para o Q6
        perda_planejada_quad = y_plan[6]
        
        # Calcular perda real no Q6
        perda_real_quad = dados_reais[6] if 6 < len(dados_reais) else 0
        
        print(f"\n📊 Status Atual: Quadrante 6 | Peso Atual: {peso_atual:.1f}kg")
        print(f"📅 Data atual: {datetime.now().strftime('%d/%m/%Y')}")
        print(f"⚠️ Desvio acumulado: {desvio:.1f}kg")
        
        # Verificar se o desvio é alto (≥25%)
        if perda_planejada_acum > 0 and abs(desvio) >= 0.25 * perda_planejada_acum:
            print("\n🚨 ALERTA: DESVIO SIGNIFICATIVO DETECTADO 🚨")
            print("="*50)
            print("ANÁLISE DETALHADA DO QUADRANTE ATUAL")
            print("="*50)
            print(f"Perda planejada acumulada: {perda_planejada_acum:.1f}kg")
            print(f"Perda real acumulada: {perda_acumulada:.1f}kg")
            print(f"Desvio acumulado: {desvio:.1f}kg ({desvio/perda_planejada_acum*100:.1f}%)")
            print("\nDETALHAMENTO DO QUADRANTE ATUAL:")
            print(f" - Planejado para este quadrante: {perda_planejada_quad:.1f}kg")
            print(f" - Realizado neste quadrante: {perda_real_quad:.1f}kg")
            if 6 < len(dados_reais):
                print(f" - Falta para meta do quadrante: {perda_planejada_quad - perda_real_quad:.1f}kg")
            print("="*50)
        
        print("\nOPÇÕES:")
        print("1. Avançar para próximo quadrante (Q7)")
        print("2. Visualizar gráfico de evolução")
        print("0. Entrar em modo emergência")
        
        opcao = int(input("\nEscolha uma opção: "))
        
        if opcao == 1:
            # Calcular automaticamente a meta para o Q7
            proximo_quadrante = 7
            meta_kg = calcular_meta_proximo_quadrante(6)  # Baseado no Q6
            
            # Calcular peso alvo
            peso_alvo = peso_atual - meta_kg
            
            res = calcular_plano(
                quadrante=proximo_quadrante,
                meta_kg=meta_kg
            )
            
            print(f"\n🔮 Planejamento para Quadrante {proximo_quadrante}")
            print(f"🎯 Meta de perda: {meta_kg:.1f}kg | Peso alvo: {peso_alvo:.1f}kg")
            
        elif opcao == 2:
            plotar_evolucao_peso()
            return
            
        elif opcao == 0:
            print("\n🚨 PROTOCOLO DE EMERGÊNCIA ATIVADO 🚨")
            hoje = datetime.now().strftime("%d/%m/%Y")
            print(f"📅 Data de início: {hoje} (hoje)")
            
            meta_kg_emergencia = float(input("Informe a meta de kg a perder: "))
            dias_emergencia = int(input("Informe o número de dias para atingir a meta: "))
            
            res = calcular_plano(
                quadrante=0,
                meta_kg=meta_kg_emergencia,
                dias=dias_emergencia,
                data_inicio_quad=datetime.now()
            )
        else:
            raise ValueError("Opção inválida. Deve ser 0, 1 ou 2.")
        
        # Exibir resultados
        print(f"\n{'='*70}")
        print(f"📊 QUADRANTE: {res['quadrante']} | FASE: {res['fase'].upper()}")
        
        if res['quadrante'] > 0:
            print(f"⚠️ Desvio acumulado: {res['desvio_kg']} kg")
        
        print(f"⏳ Dias: {res['dias']} (início: {res['inicio']})")
        print(f"🎯 Meta: {res['meta_kg']} kg | {res['total_kcal']:.0f} kcal")
        print(f"🔥 Déficit diário necessário: {res['deficit_diario']:.0f} kcal")
        print(f"🍎 Dieta Déficit base: {res['deficit_alimentacao']:.0f} kcal")
        print(f"💧 Água: {res['agua']}ml")
        
        print("\n📅 PLANO DIÁRIO:")
        for data, dia, plano, atividades_comp, exerc, aliment, total, meta in res['planos']:
            print(f"\n>>> {data} ({dia}):")
            for nome, det, kcal in plano:
                print(f" - {nome}: {det} = {kcal:.1f} kcal")
            
            # Mostrar atividades complementares se houver
            if atividades_comp:
                print("🔥🔥 Complementar")
                for nome, det, kcal in atividades_comp:
                    print(f" - {nome}: {det} = {kcal:.1f} kcal")
            
            status = "✅ ATINGIDO" if total >= meta else f"⚠️ FALTAM {meta - total:.1f} kcal"
            print(f"🍎 Dieta Déficit: {aliment:.1f} kcal")
            print(f"🔥 TOTAL: {total:.1f} kcal {status}")
            print(f"   (Exerc. {exerc:.1f} + Alim. {aliment:.1f} | Meta: {meta:.1f} kcal)")
        
    except ValueError as e:
        print(f"Erro: {e}")

In [ ]:
# ======================
# EXECUÇÃO NO JUPYTER NOTEBOOK
# ======================

# Para apenas visualizar o gráfico:
plotar_evolucao_peso()

# Para executar o programa principal no Jupyter:
executar_programa_principal()

In [ ]:
import random

def generate_workout_plan(total_R=15, total_B=45, total_C=150, days=7):
    """
    Gera um plano de treino para N dias com volumes fixos e alternância muscular
    
    Parâmetros:
    total_R (int): Total diário de corrida (km)
    total_B (int): Total diário de ciclismo (km)
    total_C (int): Total diário de calistenia (repetições)
    days (int): Número de dias a gerar
    
    Retorna:
    list: Lista de dicionários contendo sequências de treino por dia
    """
    plans = []
    
    for day in range(days):
        # Define diferentes estratégias de divisão
        r_splits = [
            [total_R],
            [total_R//2, total_R//2],
            [total_R//3, total_R//3, total_R//3],
            [total_R//4, total_R//4, total_R//2]
        ]
        
        b_splits = [
            [total_B],
            [total_B//2, total_B//2],
            [total_B//3, total_B//3, total_B//3],
            [total_B*2//3, total_B//3]
        ]
        
        # Seleciona divisões aleatórias
        r_blocks = random.choice(r_splits)
        b_blocks = random.choice(b_splits)
        
        # Calcula blocos de calistenia (1 bloco por atividade de pernas)
        c_blocks = [total_C // (len(r_blocks) + len(b_blocks))] * (len(r_blocks) + len(b_blocks))
        remainder = total_C % sum(c_blocks)
        for i in range(remainder):
            c_blocks[i] += 1
        
        # Embaralha blocos de pernas
        leg_work = [('R', val) for val in r_blocks] + [('B', val) for val in b_blocks]
        random.shuffle(leg_work)
        
        # Constrói sequência com alternância muscular
        sequence = []
        start_with_upper = random.choice([True, False])
        
        for i in range(len(leg_work)):
            if start_with_upper:
                sequence.append(f'C{c_blocks[i]}')
                sequence.append(f'{leg_work[i][0]}{leg_work[i][1]}')
            else:
                sequence.append(f'{leg_work[i][0]}{leg_work[i][1]}')
                sequence.append(f'C{c_blocks[i]}')
        
        plans.append({
            "day": day + 1,
            "sequence": sequence,
            "total_R": sum(r_blocks),
            "total_B": sum(b_blocks),
            "total_C": sum(c_blocks)
        })
    
    return plans

# Exemplo de uso para 7 dias
workout_plan = generate_workout_plan(days=7)
for day_plan in workout_plan:
    print(f"Dia {day_plan['day']}: {', '.join(day_plan['sequence'])}")
    print(f"Totais: R={day_plan['total_R']}km, B={day_plan['total_B']}km, C={day_plan['total_C']} reps\n")

In [13]:
import numpy as np
from datetime import datetime, timedelta
import json
import os

# ======================
# CONSTANTES GLOBAIS
# ======================

# GASTO CALÓRICO POR ATIVIDADE
KCAL_KARATE = 1000      # 120 minutos
KCAL_CALISTENIA = 300   # 100 repetições
KCAL_CORRIDA = 100      # por km
KCAL_BIKE = KCAL_CORRIDA / 3  # por km (3km = 100kcal)

# CONSTANTES METABÓLICAS
BASAL = 2300            # Taxa metabólica basal
KCAL_POR_KG = 7000      # 7000kcal = 1kg
AGUA_Q10 = 5000         # 5L de água no Q10
MAX_DEFICIT_ALIMENTAR = 1300  # Déficit máximo via alimentação

# ======================
# FUNÇÕES AUXILIARES
# ======================

def f(x):
    """Função de progressão da perda de peso."""
    return 0.6 * (10 ** (x / 14))

def arredondar_500(valor):
    """Arredonda valores para o múltiplo de 500 mais próximo."""
    return round(valor / 500) * 500

def configurar_quadrantes():
    """Configura dias e metas por quadrante."""
    x_points = np.arange(0, 15, 1)  # Agora só até Q14
    y_plan = f(x_points)
    
    # Ajustar soma total para ~43kg
    target_total = 43
    scale_factor = target_total / np.sum(y_plan[1:15])  # Ignora Q0
    y_plan *= scale_factor
    y_plan[0] = 0  # Quadrante 0 = 0kg
    
    # Dias por quadrante (Q0 a Q14)
    dias_por_quadrante = [0]  # Q0
    for _ in range(1, 4): dias_por_quadrante.append(5)    # Q1-Q3: verde
    for _ in range(4, 8): dias_por_quadrante.append(7)    # Q4-Q7: azul
    for _ in range(8, 15): dias_por_quadrante.append(10)  # Q8-Q14: roxo
    
    return x_points, y_plan, dias_por_quadrante

def determinar_fase(quadrante):
    """Classifica a fase do quadrante."""
    if quadrante <= 3:
        return "verde"
    elif quadrante <= 7:
        return "azul"
    return "roxo"

def calcular_atividades_base(quadrante):
    """Calcula atividades físicas base para o quadrante."""
    # Progressão em relação ao Q10
    fator_q10 = quadrante / 10.0
    
    reps = max(1, round(10 * fator_q10 * 10))  # 10 reps × quadrante
    km_corrida = max(1, round(1 * fator_q10 * 10, 1))  # 1 km × quadrante
    km_bike = max(1, round(3 * fator_q10 * 10, 1))  # 3 km × quadrante
    
    return [
        ("Calistenia", f"{reps} reps", (reps / 100) * KCAL_CALISTENIA),
        ("Corrida", f"{km_corrida} km", km_corrida * KCAL_CORRIDA),
        ("Bike", f"{km_bike} km", km_bike * KCAL_BIKE)
    ]

def ajustar_karate(dia_semana, quadrante, plano_atual, deficit_restante):
    """Adiciona sessões de karatê conforme necessário."""
    if quadrante < 3:
        return 0, plano_atual
    
    # Karatê obrigatório (seg/qua)
    if dia_semana in ["segunda", "quarta"]:
        plano_atual.append(("Karate Obrigatório", "120 min", KCAL_KARATE))
        return KCAL_KARATE, plano_atual
    
    # Karatê opcional (sexta/domingo)
    if dia_semana in ["sexta", "domingo"] and deficit_restante > 250:
        # Calcula minutos proporcionais ao déficit restante
        minutos_karate = min(120, (deficit_restante / KCAL_KARATE) * 120)
        if minutos_karate >= 30:
            kcal_karate = (minutos_karate / 120) * KCAL_KARATE
            plano_atual.append(("Karate Opcional", f"{minutos_karate:.0f} min", kcal_karate))
            return kcal_karate, plano_atual
    
    return 0, plano_atual

def carregar_dados_reais():
    """Carrega dados reais do arquivo JSON."""
    dados_padrao = {
        "peso_inicial": 112.4,
        "dados_reais": [0, 1.7, -1.3, 2, 1.5],
        "quadrante_atual": 0
    }
    
    try:
        with open('dados_reais.json', 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return dados_padrao

def calcular_desvio_acumulado(quadrante):
    """Calcula o desvio acumulado com base nos dados reais."""
    dados = carregar_dados_reais()
    peso_inicial = dados["peso_inicial"]
    dados_reais = dados["dados_reais"]
    
    # Calcular perda real acumulada
    perda_real_acumulada = sum(dados_reais)
    
    # Calcular perda planejada acumulada até o quadrante anterior
    perda_planejada_acumulada = 0
    for q in range(1, quadrante):
        if q < len(y_plan):
            perda_planejada_acumulada += y_plan[q]
    
    # Calcular desvio (diferença entre planejado e realizado)
    desvio = perda_planejada_acumulada - perda_real_acumulada
    return max(0, desvio)  # Considerar apenas desvios positivos

# ======================
# CONFIGURAÇÃO GLOBAL
# ======================

# Configura quadrantes uma vez para todo o programa
x_points, y_plan, dias_por_quadrante = configurar_quadrantes()
data_inicio_padrao = datetime(2025, 7, 20)

# ======================
# FUNÇÃO PRINCIPAL
# ======================

def calcular_plano(quadrante, meta_kg_emergencia=None, dias_emergencia=None, data_inicio_emergencia=None):
    """Calcula plano detalhado para o quadrante especificado."""
    # Configurar data de início
    if quadrante == 0 and data_inicio_emergencia:
        data_inicio_quad = datetime.strptime(data_inicio_emergencia, "%d/%m/%Y")
        dias = dias_emergencia
        meta_kg = meta_kg_emergencia
        fase = "EMERGÊNCIA"
        desvio_acumulado = 0
    else:
        if quadrante < 0 or quadrante > 14:
            raise ValueError("Quadrante deve estar entre 1-14 ou 0 para emergência")
            
        dias_anteriores = sum(dias_por_quadrante[:quadrante])
        data_inicio_quad = data_inicio_padrao + timedelta(days=dias_anteriores)
        dias = dias_por_quadrante[quadrante]
        fase = determinar_fase(quadrante)
        
        # Calcular desvio acumulado automaticamente
        desvio_acumulado = calcular_desvio_acumulado(quadrante)
        
        # Calcula meta considerando desvio acumulado
        meta_base = y_plan[quadrante]
        meta_kg = meta_base + desvio_acumulado
    
    # Cálculos de déficit
    total_kcal = meta_kg * KCAL_POR_KG
    deficit_diario = arredondar_500(total_kcal / dias)
    
    # Déficit alimentar progressivo (Q1-Q14)
    if quadrante == 0:
        deficit_alimentacao = MAX_DEFICIT_ALIMENTAR
    else:
        deficit_alimentacao = min(MAX_DEFICIT_ALIMENTAR, 
                                 (MAX_DEFICIT_ALIMENTAR / 14) * quadrante)
    
    queima_necessaria = max(0, deficit_diario - deficit_alimentacao)
    
    # Configurar datas e dias da semana
    datas = [data_inicio_quad + timedelta(days=i) for i in range(dias)]
    dias_semana = [data.strftime("%A") for data in datas]
    tradutor_dias = {
        "Monday": "segunda", "Tuesday": "terça", "Wednesday": "quarta",
        "Thursday": "quinta", "Friday": "sexta", "Saturday": "sábado",
        "Sunday": "domingo"
    }
    dias_semana = [tradutor_dias[dia] for dia in dias_semana]
    
    # Calcular plano diário
    planos = []
    for i, data in enumerate(datas):
        # 1. Atividades base
        plano_dia = calcular_atividades_base(quadrante if quadrante > 0 else 5)
        total_exerc = sum(kcal for _, _, kcal in plano_dia)
        deficit_restante = max(0, queima_necessaria - total_exerc)
        
        # 2. Ajustar karatê (se necessário)
        kcal_karate, plano_dia = ajustar_karate(
            dias_semana[i], 
            quadrante, 
            plano_dia,
            deficit_restante
        )
        total_exerc += kcal_karate
        deficit_restante = max(0, queima_necessaria - total_exerc)
        
        # 3. Ajustar bike se déficit não atingido
        if deficit_restante > 0:
            # Encontra atividade de bike para atualizar
            for idx, (nome, det, kcal) in enumerate(plano_dia):
                if nome == "Bike":
                    km_atual = float(det.split()[0])
                    km_extra = deficit_restante / KCAL_BIKE
                    km_total = km_atual + km_extra
                    plano_dia[idx] = ("Bike", f"{km_total:.1f} km", kcal + km_extra * KCAL_BIKE)
                    total_exerc += km_extra * KCAL_BIKE
                    break
        
        # 4. Calcular déficit total
        total_deficit = total_exerc + deficit_alimentacao
        
        planos.append((
            data.strftime("%d/%m/%Y"),
            dias_semana[i],
            plano_dia,
            total_exerc,
            deficit_alimentacao,
            total_deficit,
            deficit_diario
        ))

    return {
        "quadrante": quadrante,
        "fase": fase,
        "dias": dias,
        "inicio": data_inicio_quad.strftime("%d/%m/%Y"),
        "meta_kg": round(meta_kg, 2),
        "desvio_kg": round(desvio_acumulado, 2),
        "total_kcal": total_kcal,
        "deficit_diario": deficit_diario,
        "deficit_alimentacao": deficit_alimentacao,
        "queima_necessaria": queima_necessaria,
        "agua": min(int(500 * quadrante), AGUA_Q10) if quadrante > 0 else 6000,
        "planos": planos
    }

# ======================
# EXECUÇÃO PRINCIPAL
# ======================

if __name__ == "__main__":
    try:
        quadrante = int(input("\nInforme o quadrante atual (0 para emergência): "))
        
        if quadrante == 0:
            print("\n🚨 PROTOCOLO DE EMERGÊNCIA ATIVADO 🚨")
            hoje = datetime.now().strftime("%d/%m/%Y")
            print(f"📅 Data de início: {hoje} (hoje)")
            
            meta_kg_emergencia = float(input("Informe a meta de kg a perder: "))
            dias_emergencia = int(input("Informe o número de dias para atingir a meta: "))
            data_inicio_emergencia = hoje
        else:
            meta_kg_emergencia = dias_emergencia = data_inicio_emergencia = None
        
        res = calcular_plano(
            quadrante,
            meta_kg_emergencia,
            dias_emergencia,
            data_inicio_emergencia
        )
        
        # Exibir resultados
        print(f"\n{'='*70}")
        print(f"📊 QUADRANTE: {res['quadrante']} | FASE: {res['fase'].upper()}")
        print(f"⚠️ Desvio acumulado: {res['desvio_kg']} kg (calculado automaticamente)")
        print(f"⏳ Dias: {res['dias']} (início: {res['inicio']})")
        print(f"🎯 Meta ajustada: {res['meta_kg']} kg | {res['total_kcal']:.0f} kcal")
        print(f"🔥 Déficit diário necessário: {res['deficit_diario']:.0f} kcal")
        print(f"🍎 Dieta Déficit base: {res['deficit_alimentacao']:.0f} kcal")
        print(f"💧 Água: {res['agua']}ml")
        
        print("\n📅 PLANO DIÁRIO:")
        for data, dia, plano, exerc, aliment, total, meta in res['planos']:
            print(f"\n>>> {data} ({dia}):")
            for nome, det, kcal in plano:
                print(f" - {nome}: {det} = {kcal:.1f} kcal")
            
            status = "✅ ATINGIDO" if total >= meta else f"⚠️ FALTAM {meta - total:.1f} kcal"
            print(f"🍎 Dieta Déficit: {aliment:.1f} kcal")
            print(f"🔥 TOTAL: {total:.1f} kcal {status}")
            print(f"   (Exerc. {exerc:.1f} + Alim. {aliment:.1f} | Meta: {meta:.1f} kcal)")
        
    except ValueError as e:
        print(f"Erro: {e}")


Informe o quadrante atual (0 para emergência):  0



🚨 PROTOCOLO DE EMERGÊNCIA ATIVADO 🚨
📅 Data de início: 27/08/2025 (hoje)


Informe a meta de kg a perder:  7
Informe o número de dias para atingir a meta:  5



📊 QUADRANTE: 0 | FASE: EMERGÊNCIA
⚠️ Desvio acumulado: 0 kg (calculado automaticamente)
⏳ Dias: 5 (início: 27/08/2025)
🎯 Meta ajustada: 7.0 kg | 49000 kcal
🔥 Déficit diário necessário: 10000 kcal
🍎 Dieta Déficit base: 1300 kcal
💧 Água: 6000ml

📅 PLANO DIÁRIO:

>>> 27/08/2025 (quarta):
 - Calistenia: 50 reps = 150.0 kcal
 - Corrida: 5.0 km = 500.0 kcal
 - Bike: 241.5 km = 8050.0 kcal
🍎 Dieta Déficit: 1300.0 kcal
🔥 TOTAL: 10000.0 kcal ✅ ATINGIDO
   (Exerc. 8700.0 + Alim. 1300.0 | Meta: 10000.0 kcal)

>>> 28/08/2025 (quinta):
 - Calistenia: 50 reps = 150.0 kcal
 - Corrida: 5.0 km = 500.0 kcal
 - Bike: 241.5 km = 8050.0 kcal
🍎 Dieta Déficit: 1300.0 kcal
🔥 TOTAL: 10000.0 kcal ✅ ATINGIDO
   (Exerc. 8700.0 + Alim. 1300.0 | Meta: 10000.0 kcal)

>>> 29/08/2025 (sexta):
 - Calistenia: 50 reps = 150.0 kcal
 - Corrida: 5.0 km = 500.0 kcal
 - Bike: 241.5 km = 8050.0 kcal
🍎 Dieta Déficit: 1300.0 kcal
🔥 TOTAL: 10000.0 kcal ✅ ATINGIDO
   (Exerc. 8700.0 + Alim. 1300.0 | Meta: 10000.0 kcal)

>>> 30/08/

Para otimizar a preservação muscular e evitar adaptação (que reduz resultados), **não repetiria a mesma ordem todos os dias**. A sequência deve variar estrategicamente nos próximos 5 dias, mantendo os princípios:

1. **Intercalar atividades de pernas (R/B) e superiores (C)** para permitir recuperação.
2. **Alternar o tipo de estímulo** (corrida impactante vs. ciclismo contínuo).
3. **Rotacionar blocos de alta/moderada intensidade** para prevenir fadiga crônica.

DESAFIO 230x Bike

115x = [48, 30, 18, 12, 8]

72x = [32, 20, 12, 8]

43x = [24, 14, 6]

---

### Proposta para 5 dias (sempre com **R15km, B45km, C150 reps**):

*Justificativa:* Começa com C para descanso das pernas após o dia anterior. B antes de R reduz impacto articular.

#### Dia 9:
0. **C100**  
1. **B48**  
2. **R5**
3. **C50**   
5. **B30**
6. **R5**   
7. **C50**
8. **B18**  
9. **R5**
10. **B12**    
12. **C50**
13. **B8**  

*Justificativa:* Blocos curtos de B (15km) intercalados com C. R concentrado no meio (pernas frescas).

#### Dia 10:
1. **R5**  
2. **C50**  
3. **B30**  
4. **C50**  
5. **R5**  
6. **B15**  
7. **C50**  

*Justificativa:* R dividido no início/fim. B longo (30km) no centro (quando o corpo está aquecido).

#### Dia 11:
1. **C50**  
2. **B25**  
3. **C50**  
4. **R10**  
5. **C50**  
6. **B20**  

*Justificativa:* Prioriza C inicial para recuperação. R único bloco após ciclismo moderado.

---

### Princípios aplicados:
- **Variabilidade diária:** Evita adaptação muscular e reduz risco de lesões por overuse.
- **R priorizado no início/mitad**e: Quando as pernas estão menos fadigadas (exceto se o dia anterior for intenso).
- **C como "pausa ativa":** Sempre entre blocos de pernas para recuperação.
- **B como transição:** Menos impacto que R, pode ser usado após corrida.
- **Monitoramento:** Se sentir fadiga residual, inverta a ordem (ex: dia 7 pode virar dia 6).

> **Importante:** Essa variação mantém o total diário (R15/B45/C150) e o gasto de ~4500 kcal, mas redistribui o estresse muscular. Para melhores resultados, inclua 1 dia de descanso a cada 7-10 dias.